### Apply layer-parallel Torchbraid to simple UCI HAR problem
The dataset collates smartphone motion sensor data for a group of 30 volunteers while each person performed six different activites (walking, walking upstairs, walking downstairs, sitting, standing, and laying). The goal of the learning here is to predict the action based on the time-series acceleration and velocity data.

The recurrent neural network used here is a layer-parallel version of a GRU network, developed in this paper:
```
Moon, Gordon Euhyun, and Eric C. Cyr. "Parallel Training of GRU Networks with a Multi-Grid Solver for Long
Sequences." arXiv preprint arXiv:2203.04738 (2022).
```

### Preliminaries
- See `examples/mnist/start0_install_mpi_notebook.ipynb`, and `examples/mnist/start1_simple_mpi_notebook.ipynb` for setting up MPI-compatible Jupyter installation
- Recommended to start ipython cluster for default notebook settings with (assumes 4 cores)

        $ ipcluster start --n=4 --engines=mpi --profile=mpi

  or

        $ ipcluster start --n=4 --engines=MPIEngineSetLauncher --profile=mpi
        
#### Layer-parallel runs most efficiently when the 

    (number of processors)*(coarsening factor) = k*(number of ResNet Layers)
    
where `k` is some integer. This experiment is designed to satisfy this with the default parameters and four processors.

In [ ]:
# Connect to local ipython cluster.  Note, the ipcluster profile name must match the
# below named profile. Here, we use 'mpi', but you can name the cluster profile anything
from ipyparallel import Client, error
cluster = Client(profile='mpi')
print('profile:', cluster.profile)
print("IDs:", cluster.ids) 

In [ ]:
%%px
# Must point to your Torchbraid location, used for location to download MNIST dataset
torchbraid_path = "/path/to/torchbraid"

# If using Makefile Torchbraid install, must uncomment to update system path to Torchbraid 
#import sys
#sys.path.append(torchbraid_path + "/src")

In [ ]:
%%px
from __future__ import print_function

import statistics as stats
import sys
from timeit import default_timer as timer
import os

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from mpi4py import MPI

import torchbraid
import torchbraid.utils

In [ ]:
%%px
# MPI information
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
procs = comm.Get_size()

In [ ]:
%%px
example_path = torchbraid_path + '/examples/UCI_HAR_Recurrent'
dataset_path = example_path + '/UCI HAR Dataset'

# If the data hasn't been downloaded yet, download it on rank 0 only
if (rank == 0) and (not os.path.exists(dataset_path)):
  import zipfile
  import urllib
  import shutil
    
  # Link to the UCI database
  download_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.zip"
  # Download the zip
  download_filename, _ = urllib.request.urlretrieve(download_url, example_path + '/UCIHARDataset.zip')
  # Upzip the file
  with zipfile.ZipFile(download_filename, 'r') as downloaded_zip:
    downloaded_zip.extractall(path=example_path)
  # Do some cleanup
  os.remove(download_filename)
  shutil.rmtree(example_path + '/__MACOSX')

In [ ]:
%%px
def imp_gru_cell_fast(dt : float, x_red_r : torch.Tensor, x_red_z : torch.Tensor, x_red_n : torch.Tensor, 
                      h : torch.Tensor, lin_rh_W : torch.Tensor, lin_zh_W : torch.Tensor,
                      lin_nr_W : torch.Tensor, lin_nr_b : torch.Tensor) -> torch.Tensor:

  r   =    torch.sigmoid(x_red_r +     F.linear(h,lin_rh_W))
  n   =    torch.   tanh(x_red_n + r * F.linear(h,lin_nr_W, lin_nr_b))
  dtz = dt*torch.sigmoid(x_red_z +     F.linear(h,lin_zh_W))

  return torch.div(torch.addcmul(h,dtz,n),1.0+dtz)

def imp_gru_cell(dt : float, x : torch.Tensor, h : torch.Tensor,
                 lin_rx_W : torch.Tensor, lin_rx_b : torch.Tensor, lin_rh_W : torch.Tensor,
                 lin_zx_W : torch.Tensor, lin_zx_b : torch.Tensor, lin_zh_W : torch.Tensor,
                 lin_nx_W : torch.Tensor, lin_nx_b : torch.Tensor, lin_nr_W : torch.Tensor, 
                 lin_nr_b : torch.Tensor) -> torch.Tensor:

  r   =      torch.sigmoid(F.linear(x, lin_rx_W, lin_rx_b) +     F.linear(h, lin_rh_W))
  n   =      torch.   tanh(F.linear(x, lin_nx_W, lin_nx_b) + r * F.linear(h, lin_nr_W,lin_nr_b))
  dtz = dt * torch.sigmoid(F.linear(x, lin_zx_W, lin_zx_b) +     F.linear(h, lin_zh_W))

  return torch.div(torch.addcmul(h, dtz, n), 1.0 + dtz)


class ImplicitGRUBlock(nn.Module):
  def __init__(self, input_size, hidden_size):
    super(ImplicitGRUBlock, self).__init__()

    #

    self.lin_rx = [None,None]
    self.lin_rh = [None,None]
    self.lin_rx[0] = nn.Linear(input_size, hidden_size, True)
    self.lin_rh[0] = nn.Linear(hidden_size, hidden_size, False)

    self.lin_zx = [None,None]
    self.lin_zh = [None,None]
    self.lin_zx[0] = nn.Linear(input_size, hidden_size, True)
    self.lin_zh[0] = nn.Linear(hidden_size, hidden_size, False)

    self.lin_nx = [None,None]
    self.lin_nr = [None,None]
    self.lin_nx[0] = nn.Linear(input_size, hidden_size, True)
    self.lin_nr[0] = nn.Linear(hidden_size, hidden_size, True)

    #

    self.lin_rx[1] = nn.Linear(hidden_size, hidden_size, True)
    self.lin_rh[1] = nn.Linear(hidden_size, hidden_size, False)

    self.lin_zx[1] = nn.Linear(hidden_size, hidden_size, True)
    self.lin_zh[1] = nn.Linear(hidden_size, hidden_size, False)

    self.lin_nx[1] = nn.Linear(hidden_size, hidden_size, True)
    self.lin_nr[1] = nn.Linear(hidden_size, hidden_size, True)

    # record the layers so that they are handled by backprop correctly
    layers =  self.lin_rx + self.lin_rh + \
              self.lin_zx + self.lin_zh + \
              self.lin_nx + self.lin_nr
    self.lin_layers = nn.ModuleList(layers)

  def reduceX(self, x):
    x_red_r = self.lin_rx[0](x)
    x_red_z = self.lin_zx[0](x)
    x_red_n = self.lin_nx[0](x)

    return (x_red_r, x_red_z, x_red_n)

  def fastForward(self, level, tstart, tstop, x_red, h_prev):
    dt = tstop-tstart

    h_prev = h_prev[0]
    h0 = imp_gru_cell_fast(dt, *x_red, h_prev[0],
                           self.lin_rh[0].weight,
                           self.lin_zh[0].weight,
                           self.lin_nr[0].weight, self.lin_nr[0].bias)
    h1 = imp_gru_cell(dt, h0, h_prev[1],
                      self.lin_rx[1].weight, self.lin_rx[1].bias, self.lin_rh[1].weight,
                      self.lin_zx[1].weight, self.lin_zx[1].bias, self.lin_zh[1].weight,
                      self.lin_nx[1].weight, self.lin_nx[1].bias, self.lin_nr[1].weight, self.lin_nr[1].bias)

    # Note: we return a tuple with a single element
    return (torch.stack((h0, h1)), )


  def forward(self, level, tstart, tstop, x, h_prev):
    dt = tstop-tstart

    h_prev = h_prev[0]
    h0 = imp_gru_cell(dt, x, h_prev[0],
                      self.lin_rx[0].weight, self.lin_rx[0].bias, self.lin_rh[0].weight,
                      self.lin_zx[0].weight, self.lin_zx[0].bias, self.lin_zh[0].weight,
                      self.lin_nx[0].weight, self.lin_nx[0].bias, self.lin_nr[0].weight, self.lin_nr[0].bias)
    h1 = imp_gru_cell(dt, h0, h_prev[1],
                      self.lin_rx[1].weight, self.lin_rx[1].bias, self.lin_rh[1].weight,
                      self.lin_zx[1].weight, self.lin_zx[1].bias, self.lin_zh[1].weight,
                      self.lin_nx[1].weight, self.lin_nx[1].bias, self.lin_nr[1].weight, self.lin_nr[1].bias)

    # Note: we return a tuple with a single element
    return (torch.stack((h0, h1)), )

class CloseLayer(nn.Module):
  def __init__(self, hidden_size, num_classes):
    super(CloseLayer, self).__init__()
    self.fc = nn.Linear(hidden_size, num_classes)

  def forward(self, x):
    x = self.fc(x)
    return x

In [ ]:
%%px
# Parallel network class, primarily builds a RNN_Parallel network object
# num_layers: number of layers per processor
# for definitions of layer-parallel (and other parameters) see more advanced scripts and notebooks
class ParallelNet(nn.Module):
  def __init__(self,
               input_size=9,
               hidden_size=100,
               Tf=None,
               num_layers=2,
               num_classes=6,
               num_steps=32,
               max_levels=3,
               max_iters=1,
               fwd_max_iters=2,
               print_level=0,
               braid_print_level=0,
               cfactor=4,
               skip_downcycle=True):
    super(ParallelNet, self).__init__()

    self.RNN_model = ImplicitGRUBlock(input_size, hidden_size)

    if Tf == None:
      Tf = float(num_steps) * MPI.COMM_WORLD.Get_size() # when using an implicit method with GRU, 
    self.Tf = Tf
    self.dt = Tf / float(num_steps * MPI.COMM_WORLD.Get_size())

    self.parallel_rnn = torchbraid.RNN_Parallel(MPI.COMM_WORLD,
                                                self.RNN_model,
                                                num_steps,hidden_size,num_layers,
                                                Tf,
                                                max_fwd_levels=max_levels,
                                                max_bwd_levels=max_levels,
                                                max_iters=max_iters)

    if fwd_max_iters > 0:
      self.parallel_rnn.setFwdMaxIters(fwd_max_iters)

    self.parallel_rnn.setPrintLevel(print_level)

    cfactor_dict = dict()
    cfactor_dict[-1] = cfactor
    self.parallel_rnn.setCFactor(cfactor_dict)
    self.parallel_rnn.setSkipDowncycle(skip_downcycle)
    self.parallel_rnn.setNumRelax(1)            # FCF on all levels, by default
    self.parallel_rnn.setFwdNumRelax(1, level=0) # F-Relaxation on the fine grid (by default)
    self.parallel_rnn.setBwdNumRelax(0, level=0) # F-Relaxation on the fine grid (by default)

    # this object ensures that only the RNN_Parallel code runs on ranks!=0
    compose = self.compose = self.parallel_rnn.comp_op()

    self.close_rnn = compose(CloseLayer, hidden_size, num_classes)

    self.hidden_size = hidden_size
    self.num_layers = num_layers

  def forward(self, x):
    h = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
    hn = self.parallel_rnn(x, h)

    x = self.compose(self.close_rnn,hn[-1,:,:])

    return x

In [ ]:
%%px
# Train model for one epoch
# Return values: per batch losses and training times, model parameters updated in-place
def train(rank, params, model, train_loader, optimizer, epoch, compose, device):
  train_times = []
  losses = []
  model.train()
  criterion = nn.CrossEntropyLoss()
  total_time = 0.0
  for batch_idx, (data, target) in enumerate(train_loader):
    start_time = timer()
    data, taraget = data.to(device), target.to(device)
    optimizer.zero_grad()
    output = model(data)
    loss = compose(criterion, output, target)
    loss.backward()
    stop_time = timer()
    optimizer.step()

    total_time += stop_time - start_time
    train_times.append(stop_time - start_time)
    losses.append(loss.item())
    if batch_idx % 10 == 0:
      root_print(rank, 'Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tTime Per Batch {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
               100. * batch_idx / len(train_loader), loss.item(), total_time / (batch_idx + 1.0)))

  root_print(rank, 'Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tTime Per Batch {:.6f}'.format(
    epoch, (batch_idx + 1) * len(data), len(train_loader.dataset),
           100. * (batch_idx + 1) / len(train_loader), loss.item(), total_time / (batch_idx + 1.0)))
  return losses, train_times

In [ ]:
%%px
# Evaluate model on validation data
# Return: number of correctly classified test items, total number of test items, loss on test data set
def test(rank, model, test_loader, compose, device):
  model.eval()
  test_loss = 0
  correct = 0
  criterion = nn.CrossEntropyLoss()
  with torch.no_grad():
    for data,target in test_loader:
      data, target = data.to(device), target.to(device)
      output = model(data)
      test_loss += compose(criterion, output, target).item()

      if rank == 0:
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()

  test_loss /= len(test_loader.dataset)

  root_print(rank, '\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))
  return correct, len(test_loader.dataset), test_loss

# Parallel printing helper function
def root_print(rank, s):
  if rank == 0:
    print(s)

In [ ]:
%%px
#Set a few training parameters  
params = {}
params['percent_data'] = 1.0     # how much of the data to read in and use for training/testing
params['batch_size'] = 100       # input batch size for training
params['epochs'] = 5             # number of epochs to train (for quick example)
#params['epochs'] = 12           # longer training, but improved results
params['lr'] = 0.01              # learning rate
params['sequence_length'] = 128  # number of samples in a single sequence (determined by the dataset)

In [ ]:
%%px
# Use device or CPU?
use_cuda = torch.cuda.is_available()
device, host = torchbraid.utils.getDevice(comm=comm)
device = torch.device("cuda" if use_cuda else "cpu")
print(f'Run info rank: {rank}: | Device: {device} | Host: {host}')

# Set seed for reproducibility
torch.manual_seed(1)

# Compute number of sample from the sequence is assigned to each processor
local_steps = int(params['sequence_length'] / procs)
if params['sequence_length'] % procs != 0:
  root_print(rank, f"Number of processors ({procs}) must be a perfect divisor of sequence length ({params['sequence_length']}).") 

In [ ]:
%%px
# Custom DataLoader for distributing sequences across ranks
class ParallelRNNDataLoader(torch.utils.data.DataLoader):
  def __init__(self, comm, dataset, batch_size, shuffle=False):
    self.dataset = dataset
    self.shuffle = shuffle
    
    rank = comm.Get_rank()
    num_procs = comm.Get_size()

    if comm.Get_rank() == 0:
      # build a gneerator to build one master initial seed
      serial_generator = torch.Generator()
      self.initial_seed = serial_generator.initial_seed()
    else:
      self.serial_loader = None
      self.initial_seed = None

    # distribute the initial seed
    self.initial_seed = comm.bcast(self.initial_seed, root=0)

    # break up sequences
    x_block = [[] for n in range(num_procs)]
    y_block = [[] for n in range(num_procs)]
    if rank == 0:
      sz = len(dataset)
      for i in range(sz):
        x,y = dataset[i]
        x_split = torch.chunk(x, num_procs, dim=0)
        y_split = num_procs*[y]

        for p,(x_in, y_in) in enumerate(zip(x_split, y_split)):
          x_block[p].append(x_in)
          y_block[p].append(y_in)

      for p,(x, y) in enumerate(zip(x_block, y_block)):
        x_block[p] = torch.stack(x)
        y_block[p] = torch.stack(y)

    x_local = comm.scatter(x_block, root=0)
    y_local = comm.scatter(y_block, root=0)

    self.parallel_dataset = torch.utils.data.TensorDataset(x_local, y_local)

    # now setup the parallel loader
    if shuffle==True:
      parallel_generator = torch.Generator()
      parallel_generator.manual_seed(self.initial_seed)
      sampler = torch.utils.data.sampler.RandomSampler(self.parallel_dataset, generator=parallel_generator)
      torch.utils.data.DataLoader.__init__(self, self.parallel_dataset, batch_size=batch_size, sampler=sampler)
    else:
      torch.utils.data.DataLoader.__init__(self, self.parallel_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
%%px
# Read in UCI_HAR 
x_data = {}
y_data = {}

for type_str in ['train', 'test']:
  d_path = dataset_path + '/' + type_str
  i_path = d_path + '/Inertial Signals/'

  # load label data
  y = np.loadtxt('%s/y_%s.txt' % (d_path, type_str))

  # give upu on the pythonic way
  y_data[type_str] = torch.tensor([int(y[i]-1) for i in range(y.shape[0])], dtype=torch.long)

  # load feature data
  body_x = np.loadtxt('%s/body_acc_x_%s.txt' % (i_path, type_str))
  body_y = np.loadtxt('%s/body_acc_y_%s.txt' % (i_path, type_str))
  body_z = np.loadtxt('%s/body_acc_z_%s.txt' % (i_path, type_str))

  gyro_x = np.loadtxt('%s/body_gyro_x_%s.txt' % (i_path, type_str))
  gyro_y = np.loadtxt('%s/body_gyro_y_%s.txt' % (i_path, type_str))
  gyro_z = np.loadtxt('%s/body_gyro_z_%s.txt' % (i_path, type_str))

  totl_x = np.loadtxt('%s/total_acc_x_%s.txt' % (i_path, type_str))
  totl_y = np.loadtxt('%s/total_acc_y_%s.txt' % (i_path, type_str))
  totl_z = np.loadtxt('%s/total_acc_z_%s.txt' % (i_path, type_str))

  x_data[type_str] = torch.Tensor(np.stack([body_x, body_y, body_z,
                                            gyro_x, gyro_y, gyro_z,
                                            totl_x, totl_y, totl_z], axis=2))

train_set = torch.utils.data.TensorDataset(x_data['train'], y_data['train'])
test_set  = torch.utils.data.TensorDataset(x_data['test'], y_data['test'])

train_size = int(7352 * params['percent_data'])
test_size = int(2947 * params['percent_data'])
train_set = torch.utils.data.Subset(train_set, range(train_size))
test_set = torch.utils.data.Subset(test_set, range(test_size))

train_loader = ParallelRNNDataLoader(comm, dataset=train_set, batch_size=params['batch_size'], shuffle=True)
test_loader = ParallelRNNDataLoader(comm, dataset=test_set, batch_size=params['batch_size'], shuffle=False)

In [ ]:
%%px
# Create layer-parallel network
# Note this can be done on only one processor, but will be slow

model = ParallelNet(num_steps=local_steps).to(device)

# Declare optimizer
optimizer = optim.SGD(model.parameters(), lr=params['lr'], momentum=0.9)

# Carry out parallel training
batch_losses = []; batch_times = []
epoch_times = []; test_times = []
validat_correct_counts = []

for epoch in range(1, params['epochs'] + 1):
  start_time = timer()
  [losses, train_times] = train(rank=rank, params=params, model=model, train_loader=train_loader, 
                                optimizer=optimizer, epoch=epoch, compose=model.compose, device=device)
  epoch_times += [timer() - start_time]
  batch_losses += losses
  batch_times += train_times

  start_time = timer()
  validat_correct, validat_size, validat_loss = test(rank=rank, model=model, test_loader=test_loader, 
                                                     compose=model.compose, device=device)
  test_times += [timer() - start_time]
  validat_correct_counts += [validat_correct]

root_print(rank,
           f'TIME PER EPOCH: {"{:.2f}".format(stats.mean(epoch_times))} '
           f'{("(1 std dev " + "{:.2f})".format(stats.mean(epoch_times))) if len(epoch_times) > 1 else ""}')
root_print(rank,
           f'TIME PER TEST:  {"{:.2f}".format(stats.mean(test_times))} '
           f'{("(1 std dev " + "{:.2f})".format(stats.mean(test_times))) if len(test_times) > 1 else ""}')

In [ ]:
%%px
from matplotlib import pyplot as plt
%matplotlib inline

# plot the loss, validation 
root_print(rank, f'\nMin batch time:   {"{:.3f}".format(np.min(batch_times))} ')
root_print(rank, f'Mean batch time:  {"{:.3f}".format(stats.mean(batch_times))} ')


if rank == 0:
  fig, ax1 = plt.subplots()
  plt.title('Layer-parallel run with %d processors\n UCI HAR dataset'%(procs), fontsize=15)
  ax1.plot(batch_losses, color='b', linewidth=2)
  ax1.grid(True, color='k', linestyle='-', linewidth=0.4)
  ax1.set_xlabel(r"Batch number", fontsize=13)
  ax1.set_ylabel(r"Loss", fontsize=13, color='b')
  
  ax2 = ax1.twinx()
  epoch_points = np.arange(1, len(validat_correct_counts) + 1) * len(train_loader)
  validation_percentage = np.array(validat_correct_counts) / validat_size
  ax2.plot( epoch_points, validation_percentage, color='r', linestyle='dashed', linewidth=2, marker='o')
  ax2.set_ylabel(r"Validation rate", fontsize=13, color='r')
